In [1]:
# region Imports


#* --------------------------------------------------------------------------------
#* General purpose imports
#* --------------------------------------------------------------------------------
import pandas as pd
import numpy as np
from astroquery.sdss import SDSS

from scipy.stats import multivariate_normal
import scipy.interpolate as interp

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.lines as mlines  # for legend proxies
import matplotlib.ticker as ticker

import seaborn as sns

import time 
import re 
from io import StringIO


#* --------------------------------------------------------------------------------
#* Personal librairies imports
#* --------------------------------------------------------------------------------
import sys, os
src_path = os.path.abspath(os.path.join("..", "src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)
from utils import astro_utils as au
from utils import maths_utils  as mu
from utils import stats_utils  as su
from utils import graphics_utils  as gu
from utils import labels_utils  as lu
from utils import pandas_utils  as pu

#* --------------------------------------------------------------------------------
#* Global variables
#* --------------------------------------------------------------------------------
import config as co
import generate_report as report

#* --------------------------------------------------------------------------------
#* Project functions imports
#* --------------------------------------------------------------------------------
import data_loader as dl
import generate_report as report
import sSFR
import morphologies as morph




# endregion

def load_data(Verbose=False):
    Control_withAGN = dl.load_SDSS()
    CG_all = dl.import_CG(Control_withAGN)

    if Verbose:
        print(f"   {len(Control_withAGN)} galaxies loaded from the SDSS database")
        print("\n")
        for cat in CG_all.keys():
            print(f"   {len(CG_all[cat])} galaxies loaded from {cat}")


    return Control_withAGN, CG_all

def select_data(Control_withAGN, CG_all, Verbose=False):
    #* --------------------------------------------------------------------------------
    #* Selecting the CG galaxies in the z, R limits
    #* and add SDSS information
    #* --------------------------------------------------------------------------------
    CG_withAGN = {}
    for cat in CG_all.keys():
        df = CG_all[cat]
        df_sel = dl.intersect_and_enrich(df, Control_withAGN)
        CG_withAGN[cat] = df_sel
        if Verbose:
            print(f"   {cat}: {len(df_sel)} galaxies left after cuts (z between {co.Z_MIN} and {co.Z_MAX}, R < {co.R_MAX})")
    
    return CG_withAGN
    
def select_nonAGN(Control_withAGN, CG_withAGN, Verbose=False):

    Control = dl.remove_AGN(Control_withAGN)

    if Verbose:
        print(f"   {len(Control)} galaxies left after removing AGN")
    CG = {}
    for cat in CG_withAGN.keys():
        CG[cat] = dl.remove_AGN(CG_withAGN[cat])
        if Verbose:
            print(f"   {cat}: {len(CG[cat])} galaxies left after removing AGN")

    # Add names to the dataframes
    Control.name = 'Control'
    for cat in CG.keys():
        CG[cat].name = cat

    return Control, CG

# region load_data
print("Loading data")
Control_withAGN, CG_all = load_data()

print("Selecting data")
CG_withAGN = select_data(Control_withAGN, CG_all)

print("Removing AGN")
Control, CG_collection = select_nonAGN(Control_withAGN, CG_withAGN)

 # using CG_4_500 for the rest of the study
CG = CG_collection['4_500']

# endregion

Loading data
Querying the SDSS database...
69995 galaxies retrieved.
NB: physical parameters come from MPA-JHU spectroscopic catalogue
Importing CG galaxies
   5228 galaxies read from gal_in_CG_new.dat-3-dv1
      3.0e-03 <= z <= 1.3e-01
      9.70 <= r <= 17.72
   4878 galaxies read from gal_in_CG_new.dat-3-dv5
      3.0e-03 <= z <= 1.2e-01
      9.70 <= r <= 17.72
   2242 galaxies read from gal_in_CG_new.dat-4-dv1
      3.6e-03 <= z <= 1.3e-01
      10.59 <= r <= 17.69
   1888 galaxies read from gal_in_CG_new.dat-4-dv5
      3.6e-03 <= z <= 1.2e-01
      10.59 <= r <= 17.69
Done
Selecting data
Removing AGN


In [2]:
Control = morph.classify(Control)
CG = morph.classify(CG)

In [3]:
Control['morphology'].value_counts()

morphology
Spiral        34715
Elliptical    10091
Uncertain      7725
Name: count, dtype: int64

In [4]:
34715/(34715+10091)

0.7747846270588761

In [5]:
567/(567+167)

0.7724795640326976

In [6]:
CG['morphology'].value_counts()

morphology
Spiral        567
Elliptical    167
Uncertain     145
Name: count, dtype: int64

In [8]:
# import res_fisher
from scipy.stats import fisher_exact

Nb_S_Control = len(Control[Control['morphology'] == 'Spiral'])
Nb_E_Control = len(Control[Control['morphology'] == 'Elliptical'])
Nb_S_CG = len(CG[CG['morphology'] == 'Spiral'])
Nb_E_CG = len(CG[CG['morphology'] == 'Elliptical'])
fraction_S_Control = Nb_S_Control / (Nb_S_Control + Nb_E_Control)
fraction_E_Control = Nb_E_Control / (Nb_S_Control + Nb_E_Control)
fraction_S_CG = Nb_S_CG / (Nb_S_CG + Nb_E_CG)
fraction_E_CG = Nb_E_CG / (Nb_S_CG + Nb_E_CG)

table = [[Nb_S_Control, Nb_E_Control],
        [Nb_S_CG, Nb_E_CG]]
res_fisher = fisher_exact(table, alternative='two-sided')
pval_morphologies_CGvsControl = res_fisher.pvalue
print(f"Control: {100*fraction_S_Control:.1f}% Spiral, {100*fraction_E_Control:.1f}% Elliptical")
print(f"CG: {100*fraction_S_CG:.1f}% Spiral, {100*fraction_E_CG:.1f}% Elliptical")

Control: 77.5% Spiral, 22.5% Elliptical
CG: 77.2% Spiral, 22.8% Elliptical


In [10]:
Control, CG, fit_results, f_interp = sSFR.compute_status(Control, CG)

Fitting GMM to original data...
Running initialization 1/5...
  Finished with KL divergence: 3.4382
  New best result found!
Running initialization 2/5...
  Finished with KL divergence: 3.4492
Running initialization 3/5...
  Finished with KL divergence: 3.4383
Running initialization 4/5...
  Finished with KL divergence: 3.4497
Running initialization 5/5...
  Finished with KL divergence: 3.4392
Original data KL divergence: 3.4382
Fitting completed in 24.70 seconds


KeyError: "None of [Index(['lgm', 'sSFR'], dtype='object')] are in the [index]"

In [ ]:
# Quelle est la fraction de spirales chez les star forming galaxies ?
NB_S_SF_CG = len(CG[(CG['morphology'] == 'Spiral') & (CG['SFR'] > 0)])
NB_E_SF_CG = len(CG[(CG['morphology'] == 'Elliptical') & (CG['SFR'] > 0)])
NB_S_SF_Control = len(Control[(Control['morphology'] == 'Spiral') & (Control['SFR'] > 0)])
NB_E_SF_Control = len(Control[(Control['morphology'] == 'Elliptical') & (Control['SFR'] > 0)])
fraction_S_SF_CG = NB_S_SF_CG / (NB_S_SF_CG + NB_E_SF_CG)
fraction_E_SF_CG = NB_E_SF_CG / (NB_S_SF_CG + NB_E_SF_CG)

0.893700708652464

In [ ]:
CG.columns

Index(['idcg', 'ra_deg', 'dec_deg', 'r_obs', 'zcmb', 'iline', 'g_obs', 'objid',
       'specObjID', 'z', 'SFR', 'sSFR', 'lgm', 'h_alpha_eqw', 'h_beta_eqw',
       'oiii_5007_eqw', 'nii_6584_eqw', 'h_alpha_flux', 'h_beta_flux',
       'oiii_5007_flux', 'nii_6584_flux', 'p_E', 'p_S', 'log_NII_Ha',
       'log_OIII_Hb', 'is_AGN', 'morphology'],
      dtype='object')